In [1]:
import argparse
import glob
import json
import multiprocessing
import os
import random
import re
from importlib import import_module
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from dataset import MaskBaseDataset,MaskMultiLabelDataset
from loss import create_criterion
import timm
from torch import nn

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
# -- dataset
dataset_module = getattr(
    import_module("dataset"), 'MaskMultiLabelDataset'
)  # default: MaskBaseDataset
dataset = dataset_module(
    data_dir='../../../Data/train/images/',
)

In [15]:
transform_module = getattr(
        import_module("dataset"), 'BaseAugmentation'
    )  # default: BaseAugmentation
transform = transform_module(
    resize=(240,240),
    mean=dataset.mean,
    std=dataset.std,
)
dataset.set_transform(transform)

/opt/conda/lib/python3.10/site-packages/torchvision/transforms/transforms.py:332: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


In [16]:
dataset[0][-1]

(<MaskLabels.INCORRECT: 1>, <GenderLabels.MALE: 0>, <AgeLabels.MIDDLE: 1>)

In [41]:
import torch
import torch.nn as nn
from torchvision import models

class CustomEfficientNet(nn.Module):
    def __init__(self, num_classes_mask, num_classes_gender, num_classes_age):
        super(CustomEfficientNet, self).__init__()
        # EfficientNet 사전 훈련된 모델 로드
        self.base_model = models.efficientnet_b0(pretrained=True)

        # EfficientNet의 마지막 레이어 제거
        # self.base_model = nn.Sequential(*list(self.base_model.children())[:-1])

        # 분류기 모듈 추가
        self.classify_mask = nn.Linear(self.base_model.classifier[1].in_features, num_classes_mask)
        self.classify_gender = nn.Linear(self.base_model.classifier[1].in_features, num_classes_gender)
        self.classify_age = nn.Linear(self.base_model.classifier[1].in_features, num_classes_age)

    def forward(self, x):
        # 특징 추출
        x = self.base_model(x)
        x = torch.flatten(x, 1)

        # 세 개의 분류기에 대한 출력
        out1 = self.classify_mask(x)
        out2 = self.classify_gender(x)
        out3 = self.classify_age(x)

        return out1, out2, out3

# 모델 인스턴스 생성
num_classes1 = 3  # 첫 번째 분류기의 클래스 수
num_classes2 = 2  # 두 번째 분류기의 클래스 수
num_classes3 = 3  # 세 번째 분류기의 클래스 수

model = CustomEfficientNet(num_classes1, num_classes2, num_classes3)


/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [56]:
M = models.efficientnet_b0(pretrained=True)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [61]:
dataset[0][0].shape

torch.Size([3, 240, 240])

In [63]:
M(dataset[0])

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [45]:
import torch.optim as optim
# 손실 함수 및 옵티마이저 설정
criterion1 = nn.CrossEntropyLoss()
criterion2 = nn.CrossEntropyLoss()
criterion3 = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)


In [55]:
model(dataset[0])

TypeError: conv2d() received an invalid combination of arguments - got (tuple, Parameter, NoneType, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, tuple of ints padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)
 * (Tensor input, Tensor weight, Tensor bias, tuple of ints stride, str padding, tuple of ints dilation, int groups)
      didn't match because some of the arguments have invalid types: (!tuple!, !Parameter!, !NoneType!, !tuple!, !tuple!, !tuple!, int)


In [49]:
# 학습 과정 설정
def train(model, dataset, optimizer, criterion1, criterion2, criterion3, epochs):
    model.train()

    for epoch in range(epochs):
        for data, targets in dataset:
            # 데이터 준비 (실제 사용 데이터에 맞게 조정 필요)
            inputs = data
            target1, target2, target3 = targets  # 실제 타겟 데이터 형태에 따라 조정

            # Forward pass
            output1, output2, output3 = model(inputs)

            # 손실 계산
            loss1 = criterion1(output1, target1)
            loss2 = criterion2(output2, target2)
            loss3 = criterion3(output3, target3)
            total_loss = loss1 + loss2 + loss3

            # Backward pass 및 최적화
            optimizer.zero_grad()
            total_loss.backward()
            optimizer.step()

            print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss.item():.4f}")

In [53]:
dataset[-1][1]

(<MaskLabels.MASK: 0>, <GenderLabels.FEMALE: 1>, <AgeLabels.MIDDLE: 1>)

In [50]:
# 학습 실행
epochs = 10
train(model, dataset, optimizer, criterion1, criterion2, criterion3, epochs)


ValueError: expected 4D input (got 3D input)